In [1]:
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, Input
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Rescaling, Input, Flatten, Dropout, MaxPooling2D, RandomFlip, RandomRotation, RandomZoom
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
IMG_SIZE = 180

In [3]:
datagen_train = ImageDataGenerator(rescale=1./255)
datagen_test = ImageDataGenerator(rescale=1./255)


In [4]:
train_set = datagen_train.flow_from_directory("../dataset/ttte/train", batch_size=16, target_size=(IMG_SIZE, IMG_SIZE), class_mode='categorical')

Found 200 images belonging to 8 classes.


In [5]:
test_set = datagen_test.flow_from_directory("../dataset/ttte/validation", batch_size=16, target_size=(IMG_SIZE, IMG_SIZE), class_mode='categorical')

Found 40 images belonging to 8 classes.


In [6]:
train_set.class_indices

{'Edward': 0,
 'Emily': 1,
 'Gordon': 2,
 'Henry': 3,
 'James': 4,
 'Percy': 5,
 'Thomas': 6,
 'Toby': 7}

In [7]:
test_set.class_indices

{'Edward': 0,
 'Emily': 1,
 'Gordon': 2,
 'Henry': 3,
 'James': 4,
 'Percy': 5,
 'Thomas': 6,
 'Toby': 7}

In [8]:
STEP_SIZE_TRAIN = train_set.samples // train_set.batch_size
STEP_SIZE_TRAIN

12

In [9]:
STEP_SIZE_TEST = test_set.samples // test_set.batch_size
STEP_SIZE_TEST

2

In [10]:
data_augmentation = Sequential(
    [

    RandomFlip("horizontal"),
    RandomRotation(0.1),
    RandomZoom(0.1)
    ]
)
data_augmentation

<Sequential name=sequential, built=False>

In [11]:
input_shape = IMG_SIZE, IMG_SIZE, 3
base_model = MobileNetV2(
    input_shape=input_shape,
    include_top=False,  
    weights='imagenet'
)
base_model.trainable = False

base_model.summary()

C:\Users\donof\AppData\Local\Temp\ipykernel_20692\954290115.py:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(


Model: "mobilenetv2_1.00_224"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 180, 180,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 90, 90,    │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 90, 90,    │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 90, 90,    │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 90, 90,    │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 90, 90,    │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 90, 90,    │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 90, 90,    │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 90, 90,    │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 90, 90,    │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 90, 90,    │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 90, 90,    │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 91, 91,    │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 45, 45,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 45, 45,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 45, 45,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 45, 45,    │      2,304 │ block_1_depthwis

 Total params: 2,257,984 (8.61 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 2,257,984 (8.61 MB)

In [19]:
model = Sequential([
    Input(shape=(IMG_SIZE, IMG_SIZE, 3)),
    data_augmentation,
    base_model,
    Rescaling(1./255),
    Conv2D(16,(3,3), padding='same', activation = 'relu'),
    MaxPooling2D(padding='same'),
    Conv2D(32,(3,3), padding = 'same', activation = 'relu'),
    MaxPooling2D(padding='same'),
    Conv2D(64,(3,3), padding = 'same', activation = 'relu'),
    MaxPooling2D(padding='same'),
    Dropout(0.3),
    Flatten(),
    Dense(128, activation = 'relu'),
    Dropout(0.4),
    Dense(len(train_set.class_indices), activation='softmax')
])
model.summary()


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 6, 6, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_3 (Rescaling)         │ (None, 6, 6, 1280)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 6, 6, 16)       │       184,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 3, 3, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 3, 3, 32)       │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 2, 2, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 2, 2, 64)       │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 1, 1, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 1, 1, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 8)              │         1,032 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,474,808 (9.44 MB)

 Trainable params: 216,824 (846.97 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [20]:
opt = Adam(learning_rate=0.001)

callback = EarlyStopping(monitor='accuracy',
                         patience=5,
                         verbose=1,
                         mode="max",
                         baseline=0.3,
                          restore_best_weights=True,
                          start_from_epoch=15
                         )


In [21]:

model.compile(
            optimizer=opt,
              loss = CategoricalCrossentropy(from_logits = True),
              metrics=['accuracy'])

history = model.fit(train_set,
                           epochs=30,
                           steps_per_epoch=STEP_SIZE_TRAIN,
                           validation_data=test_set, 
                           validation_steps=STEP_SIZE_TEST,
                           verbose=1,
                           callbacks=[callback]
                           )


Epoch 1/30


c:\Users\donof\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\backend\tensorflow\nn.py:675: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


12/12 ━━━━━━━━━━━━━━━━━━━━ 15s 464ms/step - accuracy: 0.1467 - loss: 2.0783 - val_accuracy: 0.1875 - val_loss: 2.0782
Epoch 2/30
 1/12 ━━━━━━━━━━━━━━━━━━━━ 2s 191ms/step - accuracy: 0.1875 - loss: 2.0702

c:\Users\donof\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - accuracy: 0.1875 - loss: 2.0702 - val_accuracy: 0.2188 - val_loss: 2.0783
Epoch 3/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 226ms/step - accuracy: 0.1141 - loss: 2.0764 - val_accuracy: 0.1562 - val_loss: 2.0736
Epoch 4/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - accuracy: 0.0625 - loss: 2.0662 - val_accuracy: 0.1875 - val_loss: 2.0673
Epoch 5/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 225ms/step - accuracy: 0.2011 - loss: 2.0624 - val_accuracy: 0.2188 - val_loss: 2.0609
Epoch 6/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.1250 - loss: 2.0843 - val_accuracy: 0.2500 - val_loss: 2.0485
Epoch 7/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 219ms/step - accuracy: 0.2120 - loss: 2.0208 - val_accuracy: 0.2188 - val_loss: 1.9695
Epoch 8/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.3750 - loss: 1.9706 - val_accuracy: 0.2812 - val_loss: 1.9516
Epoch 9/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 225ms/step - accuracy: 0.3098 - loss: 1.8966 - val_accuracy: 0.2500 - val_loss

In [22]:
model.save("ttte_model.keras")